## Testing for data scrabing on Esco and Jobnet

In [ ]:
# Import magics

# !pip install selenium
# !pip install webdriver_manager
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
import re
import requests
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

output = Path(r'C:\Users\TheNixe\Desktop\KU\ISDS\ISDS_edit\Exam\output')

## Selecting data on Jobindex

In [ ]:
def extract_jobindex(page, tag):
    
    headers = {'User-Agent':'kjp538@alumni.ku.dk'}
    
    url = f"https://www.jobindex.dk/jobsoegning?page={page}&q={tag}"
    
    r = requests.get(url, headers)

    soup = BeautifulSoup(r.content.decode("utf-8"), "html.parser")
    
    return soup

def transform_jobindex(soup):
    
    divs = soup.find_all("div", class_="jobsearch-result")
    
    for item in divs:
        title = item.find_all("b")[0].text.strip()
        company = item.find_all("b")[1].text.strip()
        published_date = item.find("time").text.strip()
        summary = item.find_all("p")[1].text.strip()
        job_location = item.find_all("p")[0].text.strip()
        job_url =  item.select_one('[data-click*="u="]:has(> b)')['href']
        
        job = {
            "title" : title, 
            "company" : company,
            "published_date" : published_date,
            "summary" : summary,
            "job_location" : job_location,
            "job_url" : job_url
        }
        joblist.append(job)

    return  

joblist = []
search_list = ['psykolog', 'økonom']

for k in search_list:
    for i in range(1,1000,1):
        try: 
            soup = extract_jobindex(i, k)
            transform_jobindex(soup)
        except:
            break

    job_df = pd.DataFrame(data=joblist, columns=["title", "company", "published_date", "summary", "job_location", "job_url"])
    # job_df = job_df.set_index(, append=True).swaplevel(0,1)

job_df

## Importing skills from ESCO

In [ ]:
occupations = ['http://data.europa.eu/esco/occupation/99492920-e5a5-4dba-9e5a-93193147198c']

def fetching_occupation(uri):

    url = f'http://ec.europa.eu/esco/api/resource/occupation?uri={uri}&language=da'

    response = requests.get(url)

    result = response.json()

    
    job_title = result['title']
    # alternative_job_title = result['alternativeLabel']['da']
    # description = result['description']['da']['literal']
    job_list = []
    for i in range(1,1000,1):
        try:
            essential_skill = result['_links']['hasEssentialSkill'][i]['title']
            optional_skill = result['_links']['hasOptionalSkill'][i]['title']
        except:
            break

        job = {
        'title' : job_title,
        # 'alternative_job_title' : alternative_job_title,
        # 'description': description,
        'essential_skill':essential_skill,
        'optional_skill' : optional_skill
        }   
        job_list.append(job)

        jobs = pd.DataFrame(data=job_list, columns=['title', 'essential_skill', 'optional_skill'])

        jobs = jobs.set_index(['title'], append=True).swaplevel(0,1)

    return jobs
    


for i in occupations:
    jobs = fetching_occupation(i)

jobs

In [ ]:
url = 'http://ec.europa.eu/esco/api/resource/occupation?uri=http://data.europa.eu/esco/occupation/99492920-e5a5-4dba-9e5a-93193147198c&language=da'

response = requests.get(url)

result = response.json()

result


job_title = result['title']
alternative_job_title = result['alternativeLabel']['da']
for i in range(len(result['_links'])):
    essential_skill = result['_links']['hasEssentialSkill'][i]['title']
    optional_skill = result['_links']['hasOptionalSkill'][i]['title']
    print(job_title, alternative_job_title, essential_skill)

## Scraping KU

In [ ]:
Ku_list = ['https://studier.ku.dk/kandidat/oekonomi/faglig-profil-og-job/']

## Scraping UG

In [39]:
def UG(page, tag):
    
    headers = {'User-Agent':'kjp538@alumni.ku.dk'}
    
    url = f"https://www.ug.dk/search/{tag}?page={page}"
    
    r = requests.get(url, headers)

    soup = BeautifulSoup(r.content.decode("utf-8"), "html.parser")
    
    divs = soup.find_all("div", class_="node node-uddannelse node-teaser clearfix")

    list_of_articles = []

    for i in range(len(divs)):
        list_of_articles.append(divs[i].find('a')['href'])
    
    list_of_articles_final = []
    for link in list_of_articles:
        if '/uddannelser/' in link:
            list_of_articles_final.append(link)

    # list_of_articles = ['ug.dk' + s for s in list_of_articles]

    return list_of_articles_final


search_list = ['økonomi', 'psykologi', 'statskundskab', 'datalogi', 'antropologi']
education_url = []

for k in search_list:
    for i in range(0,20,1):
        try: 
            education_url.append({k:UG(i,k)})
        except:
            break

education_url

[{'økonomi': ['/uddannelser/bachelorogkandidatuddannelser/bacheloruddannelser/samfundsvidenskabeligebacheloruddannelser/erhvervsoekonomi/oekonomi',
   '/uddannelser/bachelorogkandidatuddannelser/kandidatuddannelser/samfundsvidenskabeligekandidatuddannelser/erhvervsoekonomi/oekonomi',
   '/uddannelser/professionsbacheloruddannelser/oekonomiskmerkantileuddannelser/oekonomi-og-informationsteknologi',
   '/uddannelser/bachelorogkandidatuddannelser/bacheloruddannelser/samfundsvidenskabeligebacheloruddannelser/erhvervsoekonomi/business-administration-and-digital-management',
   '/uddannelser/bachelorogkandidatuddannelser/bacheloruddannelser/samfundsvidenskabeligebacheloruddannelser/forvaltningmv/politik-og-oekonomi',
   '/uddannelser/bachelorogkandidatuddannelser/bacheloruddannelser/samfundsvidenskabeligebacheloruddannelser/erhvervsoekonomi/miljoe-og-foedevareoekonomi',
   '/uddannelser/bachelorogkandidatuddannelser/bacheloruddannelser/naturvidenskabeligebacheloruddannelser/matematikfysikkem

In [127]:
main_text = []
    
headers = {'User-Agent':'kjp538@alumni.ku.dk'}

url = 'http://ug.dk' + education_url[0]['økonomi'][0]

r = requests.get(url, headers)

soup = BeautifulSoup(r.content.decode("utf-8"), "html.parser")
    
divs_intro = soup.find_all("div", class_="pane-content")

temp_intro = soup.find_all('p')

divs_main = soup.find_all("div", class_="field-content")

temp_main = soup.find_all('li')

In [128]:
temp_intro = temp_intro[1:5]
temp_main = temp_main[5:10]
main_text.append(temp_main)
main_text.append(temp_intro)

In [131]:
main_text
main_text.strip()


AttributeError: 'list' object has no attribute 'strip'

In [86]:
temp

<div id="skip-link">
<a class="element-invisible element-focusable" href="#main-content">Gå til hovedindhold</a>
</div>